In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Links

#### Dice coefficient
- https://towardsdatascience.com/how-accurate-is-image-segmentation-dd448f896388

#### Keras data generators and how to use them
- https://towardsdatascience.com/keras-data-generators-and-how-to-use-them-b69129ed779c

#### How to Configure Image Data Augmentation in Keras
- https://machinelearningmastery.com/how-to-configure-image-data-augmentation-when-training-deep-learning-neural-networks/

#### Deep dive into multi-label classification..! (With detailed Case Study)
- https://towardsdatascience.com/journey-to-the-center-of-multi-label-classification-384c40229bff


In [ ]:
#_______________________________________________
# Charegement des libraries Keras 
#
from tensorflow.keras.models import Sequential
### Conv2d pour des photos - 2D si Video ou coleur alors 3D 
from  tensorflow.keras.layers import Conv2D
### Pour la phse de Pooling 2D si  couleur ou Videos alors 3D
from  tensorflow.keras.layers import MaxPooling2D
### Etape 3 - Applatir dans un vector Vertical 
from  tensorflow.keras.layers import Flatten
### Dense pour ajouter des couches connectées.
from  tensorflow.keras.layers import Dense
### Droput si necessaire
from tensorflow.keras.layers import MaxPool2D,Dropout
### import Metrics
from  tensorflow.keras.metrics import *
#import keras

In [ ]:
#from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam,RMSprop
from  tensorflow.keras.metrics import *
from tensorflow.keras import Model
#from keras import backend as 
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

## With Regularization
from tensorflow.keras import regularizers, optimizers

In [ ]:
from tensorflow.keras import Input
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import BatchNormalization
from keras.models import Model
from tensorflow.keras.layers import Activation

In [ ]:
import keras 
from keras import backend as K

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

## DICE Coefficient 
- https://towardsdatascience.com/how-accurate-is-image-segmentation-dd448f896388

In [ ]:
def dice_metric(inputs, target):
    intersection = 2.0 * (target * inputs).sum()
    union = target.sum() + inputs.sum()
    if target.sum() == 0 and inputs.sum() == 0:
        return 1.0

    return intersection / union

def dice_loss(inputs, target):
    num = target.size(0)
    inputs = inputs.reshape(num, -1)
    target = target.reshape(num, -1)
    smooth = 1.0
    intersection = (inputs * target)
    dice = (2. * intersection.sum(1) + smooth) / (inputs.sum(1) + target.sum(1) + smooth)
    dice = 1 - dice.sum() / num
    return dice

def bce_dice_loss(inputs, target):
    dicescore = dice_loss(inputs, target)
    bcescore = nn.BCELoss()
    bceloss = bcescore(inputs, target)

    return bceloss + dicescore

In [ ]:
def dice_coeff(y1, y2):
    y1 = Flatten(y1)
    y2 = Flatten(y2)
    return (2 * tf.sum(y1 * y2) + smothness) / (tf.sum(y1) + tf.sum(y2) + smothness)

def dice_coeff_loss(y1, y2):
    return -dice_coeff(y1, y2)

## IOU - Intersection over Union (Jacard Index)
- https://towardsdatascience.com/metrics-to-evaluate-your-semantic-segmentation-model-6bcb99639aa2#:~:text=Simply%20put%2C%20the%20IoU%20is,the%20image%20to%20the%20left.

In [ ]:
%%time
from keras import backend as K
def iou_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou

In [ ]:
%%time
import os, sys, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from imgaug import augmenters as iaa
from tqdm import tqdm
from tqdm import tqdm_notebook
import random

## Load datasets X_train, X_Val, X_test

In [ ]:
#30*16/60 = 8h
#x_test = np.load('../input/x-test-numpy/x_test.npy')
#x_train_sample_hflip = np.load('../input/trainvalsamplehflip/x_train_sample_hflip.npy' )
#y_train_sample = np.load('../input/trainvalsamplehflip/y_train_sample.npy' )
x_train_sample = np.load('../input/train-sample-stack/X_train_stack.npy' )
y_train_sample = np.load('../input/train-sample-stack/y_train_stack.npy' )


In [ ]:
x_val_sample  = np.load('../input/trainvalsamplehflip/x_val_sample_hflip.npy')
y_val_sample = np.load('../input/trainvalsamplehflip/y_val_sample.npy')

In [ ]:
df_train_rgby = pd.read_csv('../input/train-data-rgby/train_data_rgby.csv')

In [ ]:
pd.set_option('display.max_colwidth', None) 
df_train_rgby.head()

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
df_train_rgby = reduce_mem_usage(df_train_rgby)

In [ ]:
train_data_sample = df_train_rgby.sample(frac=0.12, replace=True, random_state = 10)
train_data_sample.shape

In [ ]:
x_train_sample.shape, y_train_sample.shape, ##x_val_sample.shape, y_val_sample.shape

In [ ]:
#mean = np.mean(x_train_sample_hflip)
#std = np.std(x_train_sample_hflip)

In [ ]:
#x_train_sample_hflip -= mean
#x_train_sample_hflip /= std

In [ ]:
import gc
%who DataFrame
#del training_set
gc.collect()

In [ ]:
#y_train_mask.shape
#np.save('./y_train_mask_all.npy', y_train_mask)
#y_train_mask_5000 = np.load('../input/np-train-5000/y_train_mask_5000.npy')

In [ ]:
#img = cv2.imread('../input/hpa-single-cell-image-classification/train/fa8b0e78-bbb1-11e8-b2ba-ac1f6b6435d0_blue.png')

In [ ]:
#('../input/hpa-single-cell-image-classification/train/fa8b0e78-bbb1-11e8-b2ba-ac1f6b6435d0_blue.png')[-7:]

In [ ]:
import tensorflow as tf

In [ ]:
inp = Input(shape = (128,128,4))
x = tf.keras.layers.BatchNormalization( momentum=0.98, epsilon=0.0001,)(inp)
x = Conv2D(128, (3, 3), padding = 'same')(x)
x = Activation('relu')(x)
x = Dropout(0.15)(x)
x = Conv2D(32, (3, 3))(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2, 2))(x)
x = Dropout(0.25)(x)
x = Conv2D(64, (3, 3), padding = 'same')(x)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(64, (3, 3))(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2, 2))(x)
x = Dropout(0.25)(x)
x = Flatten()(x)
x = Dense(32)(x)
x = Activation('relu')(x)
x = Dropout(0.2)(x)
output0 = Dense(19, activation = 'sigmoid')(x)

output1 = Dense(1, activation = 'sigmoid')(x)
output2 = Dense(1, activation = 'sigmoid')(x)
output3 = Dense(1, activation = 'sigmoid')(x)
output4 = Dense(1, activation = 'sigmoid')(x)
output5 = Dense(1, activation = 'sigmoid')(x)
output6 = Dense(1, activation = 'sigmoid')(x)
output7 = Dense(1, activation = 'sigmoid')(x)
output8 = Dense(1, activation = 'sigmoid')(x)
output9 = Dense(1, activation = 'sigmoid')(x)
output10 = Dense(1, activation = 'sigmoid')(x)
output11 = Dense(1, activation = 'sigmoid')(x)
output12 = Dense(1, activation = 'sigmoid')(x)
output13 = Dense(1, activation = 'sigmoid')(x)
output14 = Dense(1, activation = 'sigmoid')(x)
output15 = Dense(1, activation = 'sigmoid')(x)
output16 = Dense(1, activation = 'sigmoid')(x)
output17 = Dense(1, activation = 'sigmoid')(x)
output18 = Dense(1, activation = 'sigmoid')(x)

In [ ]:
model = Model(inp,[output0])

model = Model(inp,[output0,
output1,
output2,
output3,
output4,
output5,
output6,
output7,
output8,
output9,
output10,
output11,
output12,
output13,
output14,
output15,
output16,
output17,
output18])

In [ ]:
rlr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 20, verbose = 0, 
                            min_delta = 1e-4, mode = 'max')
es = EarlyStopping(monitor = 'val_loss', min_delta = 1e-4, patience = 2, mode = 'max', 
                       baseline = None, restore_best_weights = True, verbose = 0)
ckp = ModelCheckpoint('./model_100.hdf5', monitor = 'val_loss', verbose = 0, 
                        save_best_only = True, save_weights_only = False, mode = 'max')

In [ ]:
model.compile(optimizer = RMSprop(lr = 0.01, decay = 1e-6),
              loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = 1e-3), 
              #metrics = [BinaryAccuracy(name='binary_accuracy', dtype=None, threshold=0.5), Precision(name='precision'), Recall(name='recall')] 
              metrics = [tf.keras.metrics.CategoricalCrossentropy(name='categorical_crossentropy'),\
                         tf.keras.metrics.CategoricalAccuracy(name='categorical_accuracy'), \
                         #Precision(name='precision'), Recall(name='recall')
                        ]
              #metrics = ['categorical_accuracy', 'accuracy'],
             )

In [ ]:
# Show a summary of the model. Check the number of trainable parameters
model.summary()

In [ ]:
def dice_coeff(y1, y2):
    y1 = Flatten(y1)
    y2 = Flatten(y2)
    return (2 * sum(y1 * y2) + smothness) / (sum(y1) + sum(y2) + smothness)

def dice_coeff_loss(y1, y2):
    return -dice_coeff(y1, y2)

In [ ]:
def generator_wrapper(generator):
    for batch_x,batch_y in generator:
        yield (batch_x,[batch_y[:,i] for i in range(19)])

In [ ]:
#training_set = np.copy(x_train_sample)

In [ ]:
#gc.collect()

## Flows Methods
- https://medium.com/swlh/keras-imagedatagenerators-flow-methods-and-when-to-use-them-b9314489d591

## Using Flow from Dataframe

In [ ]:
train_data_sample.head()

In [ ]:
cols = ['ID','image_path','Label','Nucleoplasm', 'Nuclear membrane', 'Nucleoli', 'Nucleoli fibrillar center', 
           'Nuclear speckles', 'Nuclear bodies', 'Endoplasmic reticulum', 'Golgi apparatus', 
           'Intermediate filaments', 'Actin filaments', 'Microtubules', 'Mitotic spindle', 
           'Centrosome', 'Plasma membrane', 'Mitochondria', 'Aggresome', 'Cytosol', 
           'Vesicles and punctate cytosolic patterns', 'Negative']
type(classes)

In [ ]:
train_data_sample.columns = cols

In [ ]:
train_data_sample.columns

In [ ]:
classes = ['Nucleoplasm', 'Nuclear membrane', 'Nucleoli', 'Nucleoli fibrillar center', 
           'Nuclear speckles', 'Nuclear bodies', 'Endoplasmic reticulum', 'Golgi apparatus', 
           'Intermediate filaments', 'Actin filaments', 'Microtubules', 'Mitotic spindle', 
           'Centrosome', 'Plasma membrane', 'Mitochondria', 'Aggresome', 'Cytosol', 
           'Vesicles and punctate cytosolic patterns', 'Negative']
type(classes)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
#datagen = ImageDataGenerator(rescale=1./255.,validation_split=0.2)
datagen = ImageDataGenerator(validation_split=0.2)

train_generator = datagen.flow_from_dataframe(dataframe=train_data_sample,
                               x_col='image_path',
                               #label_mode="int"
                               y_col=classes,
                               batch_size=32,
                               shuffle=True,
                               seed=42,
                                class_mode='multi_output',
                                target_size=(32,32),
                               subset='training')

In [ ]:
%%time
##### USINg IMAGE GENERATOR with Numpy Array
#STEP_SIZE_TRAIN=training_set.n//training_set.batch_size
#STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
#STEP_SIZE_TEST=test_set.n//test_set.batch_size
hist_1 = model.fit(train_generator, 
          batch_size = 64, 
          epochs=40,
          #validation_data = (valid_generator),
          #validation_split = .3,
          shuffle=False,
          verbose = 2
         )

In [ ]:

#STEP_SIZE_TRAIN=training_set.n//training_set.batch_size
#STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
#STEP_SIZE_TEST=test_set.n//test_set.batch_size
model.fit(generator_wrapper(training_set),
                  #(np.asarray(training_set).astype("float32")),
                    #steps_per_epoch=STEP_SIZE_TRAIN,
                    #validation_data=generator_wrapper(test_set),
                    batch_size=4096, 
                    #validation_steps=STEP_SIZE_VALID,
                    validation_split = .3,
                    epochs=3,
                   #verbose=2
         )

## Flow from Numpy Array

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
#datagen = ImageDataGenerator(rescale=1./255.,validation_split=0.2)
datagen = ImageDataGenerator(validation_split=0.2)

train_generator = datagen.flow(x_train_sample,
                               y_train_sample,
                               batch_size=32,
                               color_mode = 'rgb'  ### ADDED Now
                               shuffle=True,
                               seed=42,
                               subset='training')



In [ ]:
valid_generator = datagen.flow(x_val_sample,
                               y_val_sample,
                               batch_size=32,
                               seed=42,
                               subset='validation')

In [ ]:
%%time
##### USINg IMAGE GENERATOR with Numpy Array
#STEP_SIZE_TRAIN=training_set.n//training_set.batch_size
#STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
#STEP_SIZE_TEST=test_set.n//test_set.batch_size
hist_1 = model.fit(train_generator, 
          batch_size = 64, 
          epochs=40,
          validation_data = (valid_generator),
          #validation_split = .3,
          shuffle=False,
          verbose = 2
         )

In [ ]:
%%time
##### USINg IMAGE GENERATOR with Numpy Array
#STEP_SIZE_TRAIN=training_set.n//training_set.batch_size
#STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
#STEP_SIZE_TEST=test_set.n//test_set.batch_size
hist_1 = model.fit(train_generator, 
          batch_size = 16, 
          epochs=10,
          validation_data = (valid_generator),
          #validation_split = .3,
          shuffle=False,
          verbose = 2
         )

## NO FLOW 

In [ ]:
%%time
### WITh NUMPY ARRAY 
#STEP_SIZE_TRAIN=training_set.n//training_set.batch_size
#STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
#STEP_SIZE_TEST=test_set.n//test_set.batch_size
hist_1 = model.fit(x_train_sample, y_train_sample, 
          batch_size = 16, 
          epochs=10,
          validation_data = (x_val_sample, y_val_sample),
          #validation_split = .3,
          shuffle=False,
          verbose = 2
         )

In [ ]:
import seaborn as sns
def plot_model_acc(history, model):
    plt.rcParams.update({'font.size': 16, 'font.weight':'bold'})
    sns.set_style('whitegrid')
    plt.figure(figsize=(10,6))
    plt.plot(history.history['categorical_accuracy'])
    plt.plot(history.history['val_categorical_accuracy'])
    plt.title('Model Accuracy', fontsize=22, fontweight="bold")
    plt.ylabel('Accuracy',fontsize=18, fontweight="bold")
    plt.xlabel('Epoch', fontsize=18, fontweight="bold")
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.savefig('Keras_Model_Accuracy_'+str(model)+'.png', bbox_inches = 'tight')
    plt.show()

In [ ]:
#___________________________________________
# Plot Model Accuracy
def plot_model_loss(history, model): 
    plt.rcParams.update({'font.size': 16, 'font.weight':'bold'})
    sns.set_style('whitegrid')
    plt.figure(figsize=(10,6))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss', fontsize=22, fontweight="bold")
    plt.ylabel('Loss', fontsize=18, fontweight="bold")
    plt.xlabel('Epoch', fontsize=18, fontweight="bold")
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.savefig('Keras_model_loss_'+str(model)+'.png', bbox_inches = 'tight')
    plt.show()

In [ ]:
plot_model_acc(hist_1,'model1')

In [ ]:
plot_model_loss(hist_1,'model1')

In [ ]:
plot_model_loss(hist_1,'model1')

In [ ]:
from tensorflow.keras import Model

In [ ]:
model_name = 'model1_10_hflip_sample'
print(model_name)
model.save('./'+model_name+'.hdf5')


In [ ]:
model = keras.models.load_model('./'+model_name+'.hdf5')
model.load_weights('./'+model_name+'.hdf5')

In [ ]:
%%time
y_pred = model.predict(x_val_sample)

In [ ]:
y_pred

In [ ]:
pd.DataFrame(np.asarray(y_pred))

In [ ]:
## Segmentation is calculated using IoU with a threshold of 0.6.
def clean_prediction(prediction):
    #predictions = prediction.copy()
    predictions = np.copy(prediction)
    for batch_index in range(len(predictions)):
        for class_index in range(len(predictions[batch_index])):
            predictions[batch_index][class_index] = 1 if predictions[batch_index][class_index] >= 0.45 else 0
    return np.array(predictions).astype(np.int) 

In [ ]:
preds = np.copy(y_pred)

In [ ]:
classes = ['Nucleoplasm', 'Nuclear membrane', 'Nucleoli', 'Nucleoli fibrillar center', 
           'Nuclear speckles', 'Nuclear bodies', 'Endoplasmic reticulum', 'Golgi apparatus', 
           'Intermediate filaments', 'Actin filaments', 'Microtubules', 'Mitotic spindle', 
           'Centrosome', 'Plasma membrane', 'Mitochondria', 'Aggresome', 'Cytosol', 
           'Vesicles and punctate cytosolic patterns', 'Negative']
type(classes)

In [ ]:
clean_pred = clean_prediction(preds)

In [ ]:
pd.DataFrame(clean_pred, columns=(classes))

In [ ]:
## Submissions are evaluated by computing [mAP], with the mean taken over the 19 segmentable classes of the challenge.
from sklearn.metrics import average_precision_score

In [ ]:
mAP = average_precision_score(y_val_sample, y_pred)
mAP

In [ ]:
mAP

In [ ]:
y_val[:1], y_pred[:1]

In [ ]:
average_precision_score(y_test[:1], y_pred[:1])